# Ensemble Methods


Ensemble methods workupon the principle that a group of weak learners come together to form a strong learner. The idea is to combine the predictions of multiple models to improve the overall performance. The ensemble methods are divided into two categories:
1. Bagging
2. Boosting

## Some Terminology
- Base Learner / Base Model / Base Estimator: the individual models used in ensemble methods
- Weak Learner: models that perform just above random chance (e.g. models with 50% accuracy)
- Strong Learner: models that perform well above random chance (e.g. models with 80% accuracy)

### Bias Variance Tradeoff

Principle behind many different regularisation techniques. Defined as:
- **Bias**: Average difference between the predicted values and the actual values. High bias models are oversimplified and do not capture the underlying patterns in the data. High bias = high error in training. Model predicts less accurately on seen data.
- **Variance**: Variability of the model's prediction for a given data point. High variance models are too complex and capture noise in the training data. High variance = high error in testing. Model predicts less accurately on unseen data.
- **Irreducible Error**: Error that cannot be reduced by model tuning. It is the error introduced by the noise in the data.

We can define Error as:
$Total Error = Bias^2 + Variance + Irreducible Error$

### Reason for using many models
Each model consists of numerous variables, each contributing to the total error. 

Even the same algorithm could yield models with different error rates. By combining multiple models, we can reduce the total error rate by averaging out the individual errors.

## Types of Ensemble Methods

Two types in general:
- **Sequential Ensemble Methods**: Models are generated sequentially and each model tries to correct the errors of the previous model. Examples include AdaBoost, Gradient Boosting, and XGBoost.
- **Parallel Ensemble Methods**: Models are generated in parallel and then combined. Examples include Random Forest and Bagging.

Parallel can be further divided into:
- **Homogenous**: All models are of the same type. Example: Random Forest.
- **Heterogenous**: Models are of different types. Example: Stacking.

## Techniques used in Ensemble Methods